In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.autonotebook import tqdm

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

/tmp/ipykernel_2193536/2386628013.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
pillar_df = pd.read_csv(Path("/data/dzeiberg/pillar_project/dataframe/pillar_data_clinvar38_annotated_final_v14_spliceai_032525.csv"))

/tmp/ipykernel_2193536/2161418308.py:1: DtypeWarning: Columns (4,11,13,14,15,21,22,23,34,37,38,39,40,43,44,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  pillar_df = pd.read_csv(Path("/data/dzeiberg/pillar_project/dataframe/pillar_data_clinvar38_annotated_final_v14_spliceai_032525.csv"))


In [6]:
pillar_df.simplified_consequence.value_counts()

simplified_consequence
Missense         864698
Synonymous        75764
Splice Region     49418
Nonsense          35457
Intronic          11252
Unknown            9033
Frameshift         2908
Inframe            1825
5' UTR             1316
3' UTR              813
Start lost          652
Stop lost           409
Other               169
Name: count, dtype: int64

In [7]:
# consequence_sets = [
#     {'Missense', "Synonymous"},
#     set(pillar_df.simplified_consequence.unique()),
#     {"Start loss", 'Nonsense', 'Synonymous', 'Missense', 'Frameshift', 'Inframe', 'Stop Loss'}
# ]
# pillar_subsets = [pillar_df[pillar_df.simplified_consequence.isin(consequences)] for consequences in consequence_sets]
amino_acid_assay_consequences = {'Missense','Synonymous','Nonsense'}

In [9]:
pillar_df.nucleotide_or_aa.unique()

array(['aa', 'nucleotide'], dtype=object)

In [10]:
pillar_subset = pillar_df[((pillar_df.nucleotide_or_aa == "nucleotide") | 
                          (pillar_df.simplified_consequence.isin(amino_acid_assay_consequences))) & 
                          ((pillar_df.SpliceAI_score < 0.2) | (pillar_df.SpliceAI_score.isna()))]

In [11]:
pillar_subset

,ID,Dataset,Gene,HGNC_id,Chrom,STRAND,hg19_pos,hg38_start,hg38_end,ref_allele,alt_allele,auth_transcript_id,transcript_pos,transcript_ref,transcript_alt,aa_pos,aa_ref,aa_alt,hgvs_c,hgvs_p,consequence,auth_reported_score,auth_reported_rep_score,auth_reported_func_class,auth_reported_normal_min,auth_reported_normal_max,auth_reported_abnormal_min,auth_reported_abnormal_max,splice_measure,gnomad_MAF,clinvar_sig,clinvar_star,clinvar_date_last_reviewed,nucleotide_or_aa,MaveDB URN (score set),Ensembl_transript_ID,Ref_seq_transcript_ID,Model_system,Assay_type,Phenotype_measured,Phenotype_detail,IGVF_produced,simplified_consequence,Flag,CHROM,POS,REF,ALT,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,SpliceAI_score
0,ASPA_Grønbæk-Thygesen_2024_abundance_var288098,ASPA_Grønbæk-Thygesen_2024_abundance,ASPA,756.0,17,1.0,NaN,3476163.0,3476163.0,A,G,NaN,4,NaN,NaN,2,T,A,c.4A>G,p.Thr2Ala,missense_variant,0.9031,NaN,NaN,NaN,NaN,NaN,NaN,No,0.000002,NaN,NaN,NaN,aa,urn:mavedb:00000657-a-1,ENST00000263080.3,NM_000049.4,immortalized human cells,Reporter,Fluorescence,protein stability,No,Missense,NaN,17.0,3476163.0,A,G,0.0,0.0,0.0,0.0,-29.0,-22.0,35.0,-29.0,0.0
1,ASPA_Grønbæk-Thygesen_2024_abundance_var288098,ASPA_Grønbæk-Thygesen_2024_abundance,ASPA,756.0,17,1.0,NaN,3476163.0,3476165.0,ACT,GCC,NaN,4-6,NaN,NaN,2,T,A,c.4_6delinsGCC,p.Thr2Ala,missense_variant,0.9031,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,aa,urn:mavedb:00000657-a-1,ENST00000263080.3,NM_000049.4,immortalized human cells,Reporter,Fluorescence,protein stability,No,Missense,NaN,17.0,3476163.0,A,G,0.0,0.0,0.0,0.0,-29.0,-22.0,35.0,-29.0,0.0
2,ASPA_Grønbæk-Thygesen_2024_abundance_var288098,ASPA_Grønbæk-Thygesen_2024_abundance,ASPA,756.0,17,1.0,NaN,3476163.0,3476165.0,ACT,GCA,NaN,4-6,NaN,NaN,2,T,A,c.4_6delinsGCA,p.Thr2Ala,missense_variant,0.9031,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,aa,urn:mavedb:00000657-a-1,ENST00000263080.3,NM_000049.4,immortalized human cells,Reporter,Fluorescence,protein stability,No,Missense,NaN,17.0,3476163.0,A,G,0.0,0.0,0.0,0.0,-29.0,-22.0,35.0,-29.0,0.0
3,ASPA_Grønbæk-Thygesen_2024_abundance_var288098,ASPA_Grønbæk-Thygesen_2024_abundance,ASPA,756.0,17,1.0,NaN,3476163.0,3476165.0,ACT,GCG,NaN,4-6,NaN,NaN,2,T,A,c.4_6delinsGCG,p.Thr2Ala,missense_variant,0.9031,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,aa,urn:mavedb:00000657-a-1,ENST00000263080.3,NM_000049.4,immortalized human cells,Reporter,Fluorescence,protein stability,No,Missense,NaN,17.0,3476163.0,A,G,0.0,0.0,0.0,0.0,-29.0,-22.0,35.0,-29.0,0.0
4,ASPA_Grønbæk-Thygesen_2024_abundance_var288099,ASPA_Grønbæk-Thygesen_2024_abundance,ASPA,756.0,17,1.0,NaN,3476163.0,3476164.0,AC,TG,NaN,4-5,NaN,NaN,2,T,C,c.4_5delinsTG,p.Thr2Cys,missense_variant,0.4523,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,aa,urn:mavedb:00000657-a-1,ENST00000263080.3,NM_000049.4,immortalized human cells,Reporter,Fluorescence,protein stability,No,Missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053709,PTEN_Mighell_2018_var96397,PTEN_Mighell_2018,PTEN,9588.0,10,1.0,NaN,87965467.0,87965469.0,GTC,ATT,NM_000314.6,1207-1209,NaN,NaN,403.0,V,I,c.1207_1209delinsATT,p.Val403Ile,missense_variant,-2.049575,NaN,0.0,-1.11,0.89,-5.76,-2.13,No,NaN,NaN,NaN,NaN,aa,urn:mavedb:00000054-a-1,ENST00000371953.8,NM_000314.6,yeast,Cell Viability,Cell growth,lipid phosphatase activity,No,Missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1053710,PTEN_Mighell_2018_var96399,PTEN_Mighell_2018,PTEN,9588.0,10,1.0,NaN,87965467.0,87965469.0,GTC,ATG,NM_000314.6,1207-1209,NaN,NaN,403.0,V,M,c.1207_1209delinsATG,p.Val403Met,missense_variant,0.106511,NaN,1.0,-1.11,0.89,-5.76,-2.13,No,NaN,NaN,NaN,NaN,aa,urn:mavedb:00000054-a-1,ENST00000371953.8,NM_000314.6,yeast,Cell Viability,Cell growth,lipid phosphatase activity,No,Missens

In [12]:
from pillar_project.data_utils.dataset import Scoreset

In [ ]:
scoresets = []
dataset_names = pillar_df["Dataset"].unique()

with tqdm(dataset_names) as pbar:
    for ds_name in pbar:
        pbar.set_description(ds_name)
        ss = pillar_subset[pillar_subset["Dataset"] == ds_name]
        scoresets.append(Scoreset(ss, quantile_min=0.0, quantile_max=1.0))

  0%|          | 0/86 [00:00<?, ?it/s]

In [15]:
counts = pd.DataFrame({ss.variants[0].Dataset: \
                       {sample_name : int(sample_count) for sample_name,sample_count in zip(ss.sample_names, ss.sample_counts)}\
                        for ss in scoresets} ).T.fillna(0).applymap(int)

/tmp/ipykernel_2193536/39896195.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for ss in scoresets} ).T.fillna(0).applymap(int)


In [16]:
counts

,Pathogenic/Likely Pathogenic,Benign/Likely Benign,gnomAD,Synonymous
ASPA_Grønbæk-Thygesen_2024_abundance,61,6,345,0
ASPA_Grønbæk-Thygesen_2024_toxicity,65,6,352,0
BRCA1_Adamovich_2022_Cisplatin,29,14,61,0
BRCA1_Adamovich_2022_HDR,25,18,70,0
CALM1_CALM2_CALM3_Weile_2017,12,0,27,130
CBS_Sun_2020_high_B6,19,0,18,461
CBS_Sun_2020_low_B6,15,0,15,476
CHK2_Gebbia_2024,29,2,353,305
CRX_Shepherdson_2024,12,8,349,0
F9_Popp_2025_carboxy_F9_specific,114,19,226,418


In [27]:
counts[(counts.loc[:,['Pathogenic/Likely Pathogenic','Benign/Likely Benign','gnomAD']] > 0).all(axis=1)].index.to_series().to_csv("Datasets_03262025.csv")

In [17]:
counts.to_csv("/data/dzeiberg/pillar_project/dataframe/counts_final.csv")

In [38]:
from pathlib import Path
from joblib import dump
savedir = Path("/data/dzeiberg/pillar_project/dataframe/final_scoresets/")
savedir.mkdir(exist_ok=True)
for ss in tqdm(scoresets, total=len(scoresets), leave=False):
    dump(ss, savedir / f"{ss.variants[0].Dataset}.pkl")

  0%|          | 0/86 [00:00<?, ?it/s]

In [22]:
pillar_df[(pillar_df.Dataset == "PTEN_Mighell_2018") & (pillar_df.clinvar_sig.str.contains("enign"))]

,ID,Dataset,Gene,HGNC_id,Chrom,STRAND,hg19_pos,hg38_start,hg38_end,ref_allele,alt_allele,auth_transcript_id,transcript_pos,transcript_ref,transcript_alt,aa_pos,aa_ref,aa_alt,hgvs_c,hgvs_p,consequence,auth_reported_score,auth_reported_rep_score,auth_reported_func_class,auth_reported_normal_min,auth_reported_normal_max,auth_reported_abnormal_min,auth_reported_abnormal_max,splice_measure,gnomad_MAF,clinvar_sig,clinvar_star,clinvar_date_last_reviewed,nucleotide_or_aa,MaveDB URN (score set),Ensembl_transript_ID,Ref_seq_transcript_ID,Model_system,Assay_type,Phenotype_measured,Phenotype_detail,IGVF_produced,simplified_consequence,Flag,CHROM,POS,REF,ALT,DS_AG,DS_AL,DS_DG,DS_DL,DP_AG,DP_AL,DP_DG,DP_DL,SpliceAI_score
1051903,PTEN_Mighell_2018_var94104,PTEN_Mighell_2018,PTEN,9588.0,10,1.0,NaN,87960984.0,87960984.0,C,G,NM_000314.6,892,NaN,NaN,298.0,Q,E,c.892C>G,p.Gln298Glu,missense_variant,0.305204,NaN,1.0,-1.11,0.89,-5.76,-2.13,No,0.00003,Likely benign,reviewed by expert panel,"Aug 04, 2023",aa,urn:mavedb:00000054-a-1,ENST00000371953.8,NM_000314.6,yeast,Cell Viability,Cell growth,lipid phosphatase activity,No,Missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
